# Kurly Hackerton 2022 - 나만의 상점 만들기 
## RS ENS_LinUCB algorithm tutorial 

### written by. DS 담당 이은표(Team Markurly)




In [1]:
# Load dependencies
from makurly_RS import *


# Loading Data 

In [9]:
#pw = ''
#ip = ''
#tab_name = ''
#q = "mariadb://makurly:" + pw + "@"" + ip + "/"  + tab_name + "?charset=utf8mb4" 

In [4]:
# Load data
engine = create_engine(q) # q : 보안 관계로 삭제
customer = pd.read_sql("SELECT * FROM customer", engine)
item = pd.read_sql("SELECT * FROM item", engine)
interaction = pd.read_sql("SELECT * FROM interaction", engine)
interaction_item = pd.read_sql("SELECT * FROM interaction_item", engine)

In [5]:
customer.head()

,id,name,gender,birth_date,device,main_address,detailed_address,sign_in_date
0,1,jiho,MALE,1964-01-01,iOS,서울특별시 서초구 서초대로73길 7,지하 1층,2018-01-01
1,2,pkeugine,MALE,1964-01-01,iOS,서울특별시 강남구 테헤란로 133,18층(역삼동),2018-01-01
2,3,bob,MALE,1993-01-01,iOS,서울특별시 강남구 테헤란로 133,18층(역삼동),2018-01-01
3,4,alice,MALE,1974-01-01,Android,서울특별시 강남구 테헤란로 133,18층(역삼동),2018-01-01
4,5,eve,FEMALE,1964-01-01,iOS,서울특별시 강남구 테헤란로 133,18층(역삼동),2018-01-01


In [6]:
item.head()

,id,name,category,price,img_url
0,1,소스 듬뿍 스테이크 3종,샐러드·닭가슴살,8900,https://product-image.kurly.com/product/image/...
1,2,시그니처 샐러드 4종,샐러드·닭가슴살,6500,https://product-image.kurly.com/product/image/...
2,3,냉동 닭가슴살 4종 (3개입),샐러드·닭가슴살,6990,https://product-image.kurly.com/product/image/...
3,4,닭가슴살 야채볶음밥(4개입),샐러드·닭가슴살,11600,https://product-image.kurly.com/product/image/...
4,5,에그마요 샌드위치 2종,샐러드·닭가슴살,3000,https://img-cf.kurly.com/shop/data/goods/16500...


In [7]:
interaction.head()

,id,customer_id,order_date
0,1,7,2019-12-31 17:51:00
1,2,8,2020-01-04 12:57:00
2,3,9,2020-01-02 15:57:00
3,4,10,2020-01-02 14:12:00
4,5,14,2020-01-01 07:16:00


In [8]:
interaction_item.head()

,id,quantity,interaction_id,item_id
0,1,1,1,65
1,2,1,1,135
2,3,1,1,124
3,4,1,2,97
4,5,1,2,74


# Train-test split (by current_date)

In [10]:
current_date = '2021-01-01'

# interaction -> interaction_train, interaction_test
interaction_train = interaction[interaction['order_date']< current_date].reset_index(drop=True)
interaction_test = interaction[interaction['order_date']>= current_date].reset_index(drop=True)


# interaction_item -> train, test
train_interaction_list = np.unique(interaction_train.id)
test_interaction_list = np.unique(interaction_test.id)

interaction_item_train = interaction_item[interaction_item.interaction_id.isin(train_interaction_list)]
interaction_item_test = interaction_item[interaction_item.interaction_id.isin(test_interaction_list)]


del train_interaction_list, test_interaction_list

# Fitting Baseline models on Train interactions

## Fitting matrix factorization models

Generating R(rating matrix)

In [11]:
R_train = transform_rating_mat(interaction = interaction_train, 
                     interaction_item = interaction_item_train, 
                     current_time = current_date) 



Fitting explicit model : ex1, ex2

In [12]:
ex1 = recommender_explicitMF(max_iters=15, n_factor=10, reg_lambda=10)

ex1.fit(R_train)

Fitting implicit model : im1, im2

In [13]:
im1 = recommender_implicitMF(max_iters=15, n_factor =10, alpha = 40, reg_lambda = 10)


im1.fit(R_train)


n_iter 0
n_iter 1
n_iter 2
n_iter 3
n_iter 4
n_iter 5
n_iter 6
n_iter 7
n_iter 8
n_iter 9
n_iter 10
n_iter 11
n_iter 12
n_iter 13
n_iter 14


## Fitting random recommender

In [14]:
rd1 = random_recommender()

# Experiment  & Dashboard data


### AP_calc : metric(AP, Recall) calculator from recorded experiment

In [23]:
import ml_metrics as metrics

def AP_calc(model_name, res) : 
    interaction_id_l = []
    AP5_l = []; Recall5_l = []
    for t in range(len(res)) : 
        interaction_id_t = res[t]['interaction_id']
        
        prediction_N_t = res[t]['top_N_item_id']
        actual_t = list(interaction_item.loc[interaction_item['interaction_id'] == interaction_id_t, 'item_id'].values)
        
        AP5 = metrics.apk(actual_t,prediction_N_t,5)
        Recall5 = np.mean([t in prediction_N_t for t in actual_t])
        
        interaction_id_l.append(interaction_id_t)
        AP5_l.append(AP5)
        Recall5_l.append(Recall5)
    
    return pd.DataFrame({'model':model_name ,'interaction_id': interaction_id_l, 'AP@5':AP5_l, 'Recall@5': Recall5_l })




### use only 20% of test sets to reduce time (just for now). 

In [16]:
# using 
ratio = 0.2
_ = np.random.choice(list(range(interaction_test.shape[0])), 
                 size= int(interaction_test.shape[0]*ratio), 
                 replace=False)

In [17]:
interaction_test_2 = interaction_test.iloc[_, : ].sort_values('order_date').reset_index(drop=True)



### Test baseline RSs on test set

In [20]:
ex1_res = run_simul_baseline(model = models[0], 
                   interaction = interaction_test_2, 
                   interaction_item = interaction_item_test)

im1_res = run_simul_baseline(model = models[1], 
                   interaction = interaction_test_2, 
                   interaction_item = interaction_item_test)

rd1_res = run_simul_baseline(model = models[2], 
                   interaction = interaction_test_2, 
                   interaction_item = interaction_item_test)





In [26]:
AP_ex1 = AP_calc(model_name = 'explicit', res = ex1_res)
AP_im1 = AP_calc(model_name = 'implicit', res=  im1_res)
AP_rd1 = AP_calc(model_name = 'random', res = rd1_res)

print(AP_ex1['AP@5'].mean())
print(AP_im1['AP@5'].mean())
print(AP_rd1['AP@5'].mean())

0.2885963865011143
0.08230858007004138
0.016012515918497294


### Test ENS LinUCB models with few hyperparameter settings :

- alpha : hyperparam determining degrees of exploration (larger alpha -> more exploration).

In [27]:
models = [ex1, im1, rd1]

In [28]:
ens_linucb000 = recommender_ens_linucb(models = models, 
                                    I = item.shape[0], U = customer.shape[0], 
                                    N=5, alpha = 0)
ens_linucb005 = recommender_ens_linucb(models = models, 
                                    I = item.shape[0], U = customer.shape[0], 
                                    N=5, alpha = 0.05)

ens_linucb01 = recommender_ens_linucb(models = models, 
                                    I = item.shape[0], U = customer.shape[0], 
                                    N=5, alpha = 0.1)

ens_linucb03 = recommender_ens_linucb(models = models, 
                                    I = item.shape[0], U = customer.shape[0], 
                                    N=5, alpha = 0.3)


ens_linucb05 = recommender_ens_linucb(models = models, 
                                    I = item.shape[0], U = customer.shape[0], 
                                    N=5, alpha = 0.5)

ens_linucb1 = recommender_ens_linucb(models = models, 
                                    I = item.shape[0], U = customer.shape[0], 
                                    N=5, alpha = 1)

ens_linucb3 = recommender_ens_linucb(models = models, 
                                    I = item.shape[0], U = customer.shape[0], 
                                    N=5, alpha = 3)

ens_linucb5 = recommender_ens_linucb(models = models, 
                                    I = item.shape[0], U = customer.shape[0], 
                                    N=5, alpha = 5)




In [30]:
ens_res000 = ens_linucb000.predict_topN_2(interaction = interaction_test_2, 
                                    interaction_item = interaction_item_test, 
                                    N=5)


AP_ens000 = AP_calc(model_name ='ens0.05', res = ens_res000)
AP_ens000['AP@5'].mean()

0.0021649156319643423

In [31]:
ens_res005 = ens_linucb005.predict_topN_2(interaction = interaction_test_2, 
                                    interaction_item = interaction_item_test, 
                                    N=5)




AP_ens005 = AP_calc(model_name ='ens0.05', res = ens_res005)
AP_ens005['AP@5'].mean()

0.18884859519261382

In [32]:
ens_res01 = ens_linucb01.predict_topN_2(interaction = interaction_test_2, 
                                    interaction_item = interaction_item_test, 
                                    N=5)


AP_ens01 = AP_calc(model_name ='ens0.1', res = ens_res01)
AP_ens01['AP@5'].mean()

0.22261570757720472

In [33]:
ens_res03 = ens_linucb03.predict_topN_2(interaction = interaction_test_2, 
                                    interaction_item = interaction_item_test, 
                                    N=5)


AP_ens03 = AP_calc(model_name ='ens0.3', res = ens_res03)
AP_ens03['AP@5'].mean()

0.24761451368990767

In [34]:
ens_res05 = ens_linucb05.predict_topN_2(interaction = interaction_test_2, 
                                    interaction_item = interaction_item_test, 
                                    N=5)




AP_ens05 = AP_calc(model_name ='ens0.5', res = ens_res05)
AP_ens05['AP@5'].mean()

0.2562621378541866

In [35]:
ens_res1 = ens_linucb1.predict_topN_2(interaction = interaction_test_2, 
                                    interaction_item = interaction_item_test, 
                                    N=5)



AP_ens1 = AP_calc(model_name ='ens1', res = ens_res1)
AP_ens1['AP@5'].mean()

0.2354097023241006

In [36]:
ens_res3 = ens_linucb3.predict_topN_2(interaction = interaction_test_2, 
                                    interaction_item = interaction_item_test, 
                                    N=5)


AP_ens3 = AP_calc(model_name ='ens3', res = ens_res3)
AP_ens3['AP@5'].mean()

0.13885536055396372

### Gathering Experiment results from baseline models( explit, implicit, random) & ENS LinUCBs. 

In [37]:
AP_out = pd.concat([AP_ex1, AP_im1, AP_rd1, AP_ens000, AP_ens005, AP_ens01, AP_ens03, AP_ens05, AP_ens1, AP_ens3],axis=0)

In [38]:
AP_out = AP_out.merge(interaction_test, how='left', 
             left_on = 'interaction_id', right_on = 'id').drop(['id'],axis=1)

In [39]:
# You can export AP_out to compare performances of different models

# AP_out = AP_out.merge(customer[['id','gender','birth_date','device','main_address']], how='left',left_on = 'customer_id', right_on ='id').drop('id',axis=1)
# AP_out.to_csv('AP_out.csv',encoding='utf-8', index=False)


In [40]:
# You can dump trained model for serving later
# import pickle
# filename='ens_res03.pkl'
# pickle.dump(ens_res03, open(filename, 'wb'), -1)

References :

- Li, Lihong, et al. "A contextual-bandit approach to personalized news article recommendation." Proceedings of the 19th international conference on World wide web. 2010. APA	
- Sagi, Omer, and Lior Rokach. "Ensemble learning: A survey." Wiley Interdisciplinary Reviews: Data Mining and Knowledge Discovery 8.4 (2018): e1249. APA	
- Das, Debashis, Laxman Sahoo, and Sujoy Datta. "A survey on recommendation system." International Journal of Computer Applications 160.7 (2017).
- Baltrunas, Linas, and Xavier Amatriain. "Towards time-dependant recommendation based on implicit feedback." Workshop on context-aware recommender systems (CARS’09). 2009. APA	
- Hidasi, Balázs, and Domonkos Tikk. "Fast ALS-based tensor factorization for context-aware recommendation from implicit feedback." Joint European Conference on Machine Learning and Knowledge Discovery in Databases. Springer, Berlin, Heidelberg, 2012.


